**ÉTAPE 0** : préparation des données

In [1]:
import process 
import pandas as pd

# Nail path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv'
# Jerome path : 'C:\Users\33640\OneDrive\Documents\GitHub\Portfolio_clustering_project\Data\DATA_Statapp.csv'
# Mohamed path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv'
df = pd.read_csv('/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv')

# Apply conversion function to 'open' and 'close' columns
df['open'] = df['open'].apply(process.safe_literal_eval)
df['close'] = df['close'].apply(process.safe_literal_eval)

# Calculate returns for each line
df['return'] = df.apply(lambda row: [(close - open) / open for open, close in zip(row['open'], row['close'])], axis=1)

new_df = df[['ticker', 'return']] # create a new data frame with the column ticker and return 

# Créons le DataFrame à partir des listes dans 'return'
# On suppose ici que 'new_df' est déjà défini et contient la colonne 'return'

# Convertir chaque liste dans la colonne 'return' en plusieurs colonnes dans le nouveau DataFrame
returns_df = pd.DataFrame(new_df['return'].tolist())

# Ajouter la colonne 'ticker' du 'new_df' au début de 'returns_df'
returns_df.insert(0, 'ticker', new_df['ticker'])

# Renommer les colonnes pour refléter qu'elles sont des rendements
returns_df.columns = ['ticker'] + [f'return_{i}' for i in range(len(returns_df.columns) - 1)]

df_cleaned = process.remove_rows_with_nan(returns_df)
df_cleaned.reset_index(drop=True, inplace=True)

process.check_nan_inf(df_cleaned)

W = process.training_phase(lookback_window=30, df_cleaned=df_cleaned)

There are no NaN values in the dataframe


UnboundLocalError: cannot access local variable 'correlation_matrix' where it is not associated with a value

**ÉTAPE 1** : Phase d'entraînement

1. Obtention de la matrice de corrélation des actifs sur une fenêtre arrière de 30 jours (1 mois)

In [11]:
lookback_window = 30
correlation_matrix = process.correlation_matrix(lookback_window, df_cleaned)

## PROBLÈME DES ARRONDIS

cluster_composition = process.cluster_composition_and_centroid(df_cleaned=df_cleaned, correlation_matrix=correlation_matrix, number_of_clusters=20, lookback_window=30)

## poids très proches ... ==> dû au fait qu'on regarde sur un trop petit échantillon (30 jours) ? 

constituent_weights = process.constituent_weights(df_cleaned=df_cleaned, cluster_composition=cluster_composition, sigma=10, lookback_window=30)

cluster_return = process.cluster_return(constituent_weights=constituent_weights, df_cleaned=df_cleaned, lookback_window=30) 

markowitz_weight = process.makowitz_weights(cluster_return=cluster_return, lookback_window=30)

W = process.final_weights(markowitz_weights=markowitz_weight, constituent_weights=constituent_weights)

/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/process.py:45: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  A_pos = mat.applymap(lambda x: x if x >= 0 else 0)
/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/process.py:46: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  A_neg = mat.applymap(lambda x: abs(x) if x < 0 else 0)
/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [3]:
W = process.training_phase(lookback_window=30, df_cleaned=df_cleaned)

UnboundLocalError: cannot access local variable 'df_cleaned' where it is not associated with a value

2. Obtention de la composition de chaque cluster et du centroïde de chacun d'entre eux

In [3]:
## PROBLÈME DES ARRONDIS

cluster_composition = process.cluster_composition_and_centroid(df_cleaned=df_cleaned, correlation_matrix=correlation_matrix, number_of_clusters=20, lookback_window=30)

/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/process.py:45: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  A_pos = mat.applymap(lambda x: x if x >= 0 else 0)
/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/process.py:46: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  A_neg = mat.applymap(lambda x: abs(x) if x < 0 else 0)
/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


**ÉTAPE 2** : construction de portefeuille

1. On donne, au sein d'un même cluster, un poids à chaque actif selon sa distance au centroïde de celui-ci. Cela nous servira plus tard pour calculer le rendement de chaque cluster (alors vu comme un nouvel actif synthétique)

In [4]:
constituent_weights = process.constituent_weights(df_cleaned=df_cleaned, cluster_composition=cluster_composition, sigma=10, lookback_window=30)

In [5]:
## poids très proches ... ==> dû au fait qu'on regarde sur un trop petit échantillon (30 jours) ? 

constituent_weights


[['cluster 1',
  [['AES', 0.968978879935598],
   ['AIR', 0.9705372363030682],
   ['APH', 0.9721267812248616],
   ['ATO', 0.9690510211614523],
   ['AVT', 0.9692294151315137],
   ['AWR', 0.9689191076100966],
   ['B', 0.9687779150342414],
   ['BRC', 0.9693038150944545],
   ['CLB', 0.9700242247875388],
   ['CRY', 0.9696362115838891],
   ['DE', 0.9691570136860282],
   ['DY', 0.9700051403124131],
   ['EMF', 0.9689317804613787],
   ['GLW', 0.97197964699841],
   ['KEP', 0.9696499191487619],
   ['LH', 0.9698793822891143],
   ['MTX', 0.969043000440281],
   ['NL', 0.968753877382471],
   ['NSL', 0.9682223546431213],
   ['PKI', 0.9696135125263295],
   ['ROP', 0.9700122478116358],
   ['RS', 0.9706591062205201],
   ['RSG', 0.9695834194266121],
   ['RVT', 0.9680225536856387],
   ['SKX', 0.9724838829195663],
   ['WGO', 0.9698538412118861],
   ['WST', 0.9686872790453006],
   ['WWW', 0.9726186056484651]]],
 ['cluster 2',
  [['AFG', 0.9109571475579223],
   ['AIG', 0.9118286923914753],
   ['ALK', 0.9105194

Le choix des rendements attendus (expected_returns) dans le modèle de Markowitz peut être un défi car il nécessite des prévisions pour chaque actif inclus dans le portefeuille. 

In [6]:
cluster_return = process.cluster_return(constituent_weights=constituent_weights, df_cleaned=df_cleaned, lookback_window=30) 

markowitz_weight = process.makowitz_weights(cluster_return=cluster_return, lookback_window=30)

W = process.final_weights(markowitz_weights=markowitz_weight, constituent_weights=constituent_weights)

In [8]:
W

[['AES', 0.968978879935598],
 ['AIR', 0.9705372363030682],
 ['APH', 0.9721267812248616],
 ['ATO', 0.9690510211614523],
 ['AVT', 0.9692294151315137],
 ['AWR', 0.9689191076100966],
 ['B', 0.9687779150342414],
 ['BRC', 0.9693038150944545],
 ['CLB', 0.9700242247875388],
 ['CRY', 0.9696362115838891],
 ['DE', 0.9691570136860282],
 ['DY', 0.9700051403124131],
 ['EMF', 0.9689317804613787],
 ['GLW', 0.97197964699841],
 ['KEP', 0.9696499191487619],
 ['LH', 0.9698793822891143],
 ['MTX', 0.969043000440281],
 ['NL', 0.968753877382471],
 ['NSL', 0.9682223546431213],
 ['PKI', 0.9696135125263295],
 ['ROP', 0.9700122478116358],
 ['RS', 0.9706591062205201],
 ['RSG', 0.9695834194266121],
 ['RVT', 0.9680225536856387],
 ['SKX', 0.9724838829195663],
 ['WGO', 0.9698538412118861],
 ['WST', 0.9686872790453006],
 ['WWW', 0.9726186056484651],
 ['AFG', 0.0],
 ['AIG', 0.0],
 ['ALK', 0.0],
 ['APD', 0.0],
 ['AXP', 0.0],
 ['AZN', 0.0],
 ['BAC', 0.0],
 ['BK', 0.0],
 ['BLK', 0.0],
 ['BMO', 0.0],
 ['BTO', 0.0],
 ['CB', 